![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [100]:
import pandas as pd
import numpy as np

# Start coding here...

In [101]:
df = pd.read_csv("bank_marketing.csv")
print(df.columns.tolist())

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage', 'month', 'day', 'contact_duration', 'number_contacts', 'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx', 'euribor_three_months', 'campaign_outcome']
credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


Defining client.csv

In [102]:

df_client = df[['client_id', 'age', 'job', 'marital','education','credit_default','mortgage']]   
#print(df_client)
df_client['job'] = df_client['job'].replace(r'\.','_',regex=True)
df_client['education'] = df_client['education'].replace(r'\.','_',regex=True)
df_client['education'] = df_client['education'].replace('unknown',np.nan)
df_client['credit_default'] = (df_client['credit_default'] == 'yes').astype(bool)
df_client['mortgage'] = (df_client['mortgage'] == 'yes').astype(bool)

#for col in [ 'job', 'education', 'credit_default','mortgage']:
#    print(col)
#    print("--------------")
#    print(df_client[col].value_counts())
#print(df_client)

df_client.to_csv('client.csv', index=False)



Defining campaign.csv

In [103]:
df_campaign = df[['client_id' , 'number_contacts' ,'contact_duration', 'previous_campaign_contacts','previous_outcome','campaign_outcome' , 'month','day']]   

#for col in [ 'previous_outcome', 'campaign_outcome','month']:
#    print(col)
#    print("--------------")
#    print(df_campaign[col].value_counts())
#print(df_campaign)

df_campaign['previous_outcome'] = (df_campaign['previous_outcome'] == 'success').astype(bool)
df_campaign['campaign_outcome'] = (df_campaign['campaign_outcome'] == 'yes').astype(bool)
# Map month abbreviations to numeric values
month_mapping = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06',
                 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}
df_campaign['month'] = df_campaign['month'].map(month_mapping)

df_campaign['last_contact_date'] = pd.to_datetime(dict(year='2022', month =df_campaign['month'], day = df_campaign['day'])).dt.strftime('%Y-%m-%d')


#for col in [ 'previous_outcome', 'campaign_outcome', 'month']:
#    print(col)
#    print("--------------")
#    print(df_campaign[col].value_counts())
#print(df_campaign)

df_campaign = df_campaign[['client_id' , 'number_contacts' ,'contact_duration', 'previous_campaign_contacts','previous_outcome','campaign_outcome' , 'last_contact_date']] 

df_campaign.to_csv('campaign.csv', index=False)


Defining economics.csv

In [104]:

df_economics = df[['client_id', 'cons_price_idx','euribor_three_months']]   
df_economics.to_csv('economics.csv', index=False)
